In [1]:
import sqlite3
import os
import dspy
import json
import argparse
from flask import Flask, render_template, request, redirect, url_for, jsonify, session
from datetime import datetime
from typing import List, Dict, Any, Optional

In [2]:
class ExerciseDB:
    def __init__(self, db_path='data/exercises.db'):
        """Initialize the database connection."""
        self.conn = sqlite3.connect(db_path)
        self.conn.row_factory = sqlite3.Row
        self.cursor = self.conn.cursor()
    
    def get_all_exercises(self):
        """Get all exercises from the database."""
        self.cursor.execute("SELECT * FROM exercises")
        return [dict(row) for row in self.cursor.fetchall()]
    
    def get_exercises_by_muscle_group(self, muscle_group):
        """Get exercises for a specific muscle group."""
        self.cursor.execute("SELECT * FROM exercises WHERE muscle_group = ?", (muscle_group,))
        return [dict(row) for row in self.cursor.fetchall()]
    
    def get_exercises_by_equipment(self, equipment):
        """Get exercises for specific equipment."""
        self.cursor.execute("SELECT * FROM exercises WHERE equipment = ?", (equipment,))
        return [dict(row) for row in self.cursor.fetchall()]
    
    def close(self):
        """Close the database connection."""
        self.conn.close()

class GymDB:
    def __init__(self, db_path='data/gyms.db'):
        """Initialize the gym database connection."""
        # Create the data directory if it doesn't exist
        os.makedirs(os.path.dirname(db_path), exist_ok=True)
        
        self.conn = sqlite3.connect(db_path)
        self.conn.row_factory = sqlite3.Row
        self.cursor = self.conn.cursor()
        self._create_tables()
    
    def _create_tables(self):
        """Create the necessary tables if they don't exist."""
        # Create gyms table
        self.cursor.execute('''
        CREATE TABLE IF NOT EXISTS gyms (
            id INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            location TEXT,
            description TEXT
        )
        ''')
        
        # Create equipment table
        self.cursor.execute('''
        CREATE TABLE IF NOT EXISTS equipment (
            id INTEGER PRIMARY KEY,
            gym_id INTEGER NOT NULL,
            name TEXT NOT NULL,
            category TEXT NOT NULL,
            quantity INTEGER DEFAULT 1,
            description TEXT,
            FOREIGN KEY (gym_id) REFERENCES gyms (id)
        )
        ''')
        
        self.conn.commit()
    
    def add_gym(self, name: str, location: str = None, description: str = None) -> int:
        """Add a new gym to the database."""
        self.cursor.execute(
            'INSERT INTO gyms (name, location, description) VALUES (?, ?, ?)',
            (name, location, description)
        )
        self.conn.commit()
        return self.cursor.lastrowid
    
    def add_equipment(self, gym_id: int, name: str, category: str, 
                     quantity: int = 1, description: str = None) -> int:
        """Add equipment to a gym."""
        self.cursor.execute(
            'INSERT INTO equipment (gym_id, name, category, quantity, description) VALUES (?, ?, ?, ?, ?)',
            (gym_id, name, category, quantity, description)
        )
        self.conn.commit()
        return self.cursor.lastrowid
    
    def get_gym(self, gym_id: int) -> Optional[Dict]:
        """Get gym details by ID."""
        self.cursor.execute('SELECT * FROM gyms WHERE id = ?', (gym_id,))
        gym = self.cursor.fetchone()
        if gym:
            return dict(gym)
        return None
    
    def get_all_gyms(self) -> List[Dict]:
        """Get all gyms."""
        self.cursor.execute('SELECT * FROM gyms ORDER BY name')
        return [dict(row) for row in self.cursor.fetchall()]
    
    def get_gym_equipment(self, gym_id: int) -> List[Dict]:
        """Get all equipment for a specific gym."""
        self.cursor.execute('SELECT * FROM equipment WHERE gym_id = ? ORDER BY category, name', (gym_id,))
        return [dict(row) for row in self.cursor.fetchall()]
    
    def close(self):
        """Close the database connection."""
        self.conn.close()

# DSPy Classes for Workout Generation
class Exercise(dspy.Signature):
    """Information about an exercise."""
    id: int = dspy.OutputField()
    name: str = dspy.OutputField()
    muscle_group: str = dspy.OutputField()
    equipment: str = dspy.OutputField()

class WorkoutRequest(dspy.Signature):
    """A request for a workout."""
    workout_request: str = dspy.InputField()
    gym_equipment: List[Dict] = dspy.InputField()
    available_exercises: List[Dict] = dspy.InputField()
    title: str = dspy.OutputField()
    description: str = dspy.OutputField()
    exercises: List[Dict[str, Any]] = dspy.OutputField()
    sets_and_reps: List[str] = dspy.OutputField()
    rest_times: List[str] = dspy.OutputField()
    notes: str = dspy.OutputField()
    

class WorkoutGenerator(dspy.Module):
    """Module to generate a workout plan based on user input and available equipment."""
    
    def __init__(self):
        super().__init__()
        self.generate_workout = dspy.ChainOfThought(
            WorkoutRequest
        )
        # Use ExerciseDB directly instead of a retriever
        self.exercise_db = ExerciseDB()
    
    def forward(self, description: str, gym_equipment: List[Dict]):
        """Generate a workout plan based on user description and gym equipment."""
        # Find relevant exercises using direct database queries
        # Extract potential muscle groups and equipment from description
        relevant_exercises = self.find_exercises_for_workout(description)

        print(description)
        # Generate the workout plan
        workout_request = WorkoutRequest(
            workout_request=description,
            gym_equipment=gym_equipment,
            available_exercises=relevant_exercises
        )
        workout_plan = self.generate_workout(workout_request)
        
        return workout_plan
    
    def find_exercises_for_workout(self, description):
        """Find exercises that match the workout description."""
        # Extract potential muscle groups and equipment from description
        muscle_groups = self._extract_muscle_groups(description)
        equipment = self._extract_equipment(description)
        
        # Build query based on extracted terms
        params = []
        conditions = []
        
        if muscle_groups:
            placeholders = ', '.join(['?'] * len(muscle_groups))
            conditions.append(f"muscle_group IN ({placeholders})")
            params.extend(muscle_groups)
        
        if equipment:
            placeholders = ', '.join(['?'] * len(equipment))
            conditions.append(f"equipment IN ({placeholders})")
            params.extend(equipment)
        
        # If no specific conditions, return a diverse set
        if not conditions:
            # Get a variety of exercises across different muscle groups
            return self._get_diverse_exercise_set()
        
        # Execute the query
        query = f"SELECT * FROM exercises WHERE {' OR '.join(conditions)}"
        self.exercise_db.cursor.execute(query, params)
        return [dict(row) for row in self.exercise_db.cursor.fetchall()]
    
    def _get_diverse_exercise_set(self, limit_per_group=3):
        """Get a diverse set of exercises covering different muscle groups."""
        # Get distinct muscle groups
        self.exercise_db.cursor.execute("SELECT DISTINCT muscle_group FROM exercises")
        muscle_groups = [row[0] for row in self.exercise_db.cursor.fetchall()]
        
        # Get exercises for each muscle group
        results = []
        for group in muscle_groups:
            self.exercise_db.cursor.execute(
                "SELECT * FROM exercises WHERE muscle_group = ? LIMIT ?", 
                (group, limit_per_group)
            )
            results.extend([dict(row) for row in self.exercise_db.cursor.fetchall()])
        
        return results
    
    def _extract_muscle_groups(self, description):
        """Extract potential muscle groups from a description."""
        common_muscle_groups = [
            "Chest", "Back", "Legs", "Shoulders", "Arms", 
            "Biceps", "Triceps", "Abs", "Core", "Glutes", 
            "Quads", "Hamstrings", "Calves"
        ]
        
        # Find mentioned muscle groups
        found_groups = []
        description_lower = description.lower()
        
        for group in common_muscle_groups:
            if group.lower() in description_lower:
                found_groups.append(group)
        
        return found_groups
    
    def _extract_equipment(self, description):
        """Extract potential equipment from a description."""
        common_equipment = [
            "Barbell", "Dumbbells", "Machine", "Cable", "Bodyweight",
            "Kettlebell", "Resistance Band", "Smith Machine", "TRX"
        ]
        
        # Find mentioned equipment
        found_equipment = []
        description_lower = description.lower()
        
        for equip in common_equipment:
            if equip.lower() in description_lower:
                found_equipment.append(equip)
        
        return found_equipment

# No longer using setup_vector_db - directly querying SQLite instead

def configure_lm(provider='openai'):
    """Configure the language model based on provider."""
    if provider.lower() == 'claude':
        # Set up Anthropic Claude
        api_key = os.environ.get('ANTHROPIC_API_KEY')
        if not api_key:
            raise ValueError("ANTHROPIC_API_KEY environment variable not found")
        
        return dspy.LM('anthropic/claude-3-opus-20240229', api_key=api_key)
    else:
        # Default to OpenAI
        api_key = os.environ.get('OPENAI_API_KEY')
        if not api_key:
            raise ValueError("OPENAI_API_KEY environment variable not found")
        
        # In dspy v2.0.0+, use ChatOpenAI instead of OpenAI
        try:
            return dspy.LM('openai/gpt-4o-mini', api_key=api_key)
        except AttributeError:
            # Fallback for compatibility with different dspy versions
            import openai
            openai.api_key = api_key
            return dspy.LM('openai/gpt-4o-mini', api_key=api_key)

def bootstrap_examples():
    """Create examples for bootstrapping."""
    examples = [
        dspy.Example(
            WorkoutRequest(
                workout_request="I want a quick full body workout with dumbbells",
                gym_equipment=[
                    {"name": "Dumbbells", "category": "Free Weights", "quantity": 10},
                    {"name": "Bench", "category": "Free Weights", "quantity": 2}
                ],
                available_exercises=[  # Adding the missing required field
                    {"name": "Dumbbell Squat", "muscle_group": "Legs", "equipment": "Dumbbells"},
                    {"name": "Dumbbell Bench Press", "muscle_group": "Chest", "equipment": "Dumbbells"},
                    {"name": "Dumbbell Row", "muscle_group": "Back", "equipment": "Dumbbells"},
                    {"name": "Lateral Raise", "muscle_group": "Shoulders", "equipment": "Dumbbells"},
                    {"name": "Bicep Curl", "muscle_group": "Arms", "equipment": "Dumbbells"},
                    {"name": "Overhead Tricep Extension", "muscle_group": "Arms", "equipment": "Dumbbells"},
                ],
                title="Quick Full Body Dumbbell Workout",
                description="A time-efficient full body workout using only dumbbells, perfect for building strength and endurance.",
                exercises=[
                    {"name": "Dumbbell Squat", "muscle_group": "Legs", "equipment": "Dumbbells", "sets": 3, "reps": 12},
                    {"name": "Dumbbell Bench Press", "muscle_group": "Chest", "equipment": "Dumbbells", "sets": 3, "reps": 12},
                    {"name": "Dumbbell Row", "muscle_group": "Back", "equipment": "Dumbbells", "sets": 3, "reps": 12},
                    {"name": "Lateral Raise", "muscle_group": "Shoulders", "equipment": "Dumbbells", "sets": 3, "reps": 12},
                    {"name": "Bicep Curl", "muscle_group": "Arms", "equipment": "Dumbbells", "sets": 3, "reps": 12},
                    {"name": "Overhead Tricep Extension", "muscle_group": "Arms", "equipment": "Dumbbells", "sets": 3, "reps": 12},
                ],
                sets_and_reps=["3 sets of 12 reps for each exercise"],
                rest_times=["60 seconds between sets", "90 seconds between exercises"],
                notes="Start with a 5-minute warm-up. Use a weight that challenges you by the last rep. Focus on proper form rather than heavy weight."
            )
        ),
        dspy.Example(
            WorkoutRequest(
                workout_request="I want a quick full body workout with dumbbells",
                gym_equipment=[
                    {"name": "Dumbbells", "category": "Free Weights", "quantity": 10},
                    {"name": "Bench", "category": "Free Weights", "quantity": 2}
                ],
                available_exercises=[  # Add this missing field
                    {"name": "Dumbbell Squat", "muscle_group": "Legs", "equipment": "Dumbbells"},
                    {"name": "Dumbbell Bench Press", "muscle_group": "Chest", "equipment": "Dumbbells"},
                    {"name": "Dumbbell Row", "muscle_group": "Back", "equipment": "Dumbbells"},
                    {"name": "Lateral Raise", "muscle_group": "Shoulders", "equipment": "Dumbbells"},
                    {"name": "Bicep Curl", "muscle_group": "Arms", "equipment": "Dumbbells"},
                    {"name": "Overhead Tricep Extension", "muscle_group": "Arms", "equipment": "Dumbbells"},
                ],
                title="Quick Full Body Dumbbell Workout",
                description="A time-efficient full body workout using only dumbbells, perfect for building strength and endurance.",
                exercises=[
                    {"name": "Dumbbell Squat", "muscle_group": "Legs", "equipment": "Dumbbells", "sets": 3, "reps": 12},
                    {"name": "Dumbbell Bench Press", "muscle_group": "Chest", "equipment": "Dumbbells", "sets": 3, "reps": 12},
                    {"name": "Dumbbell Row", "muscle_group": "Back", "equipment": "Dumbbells", "sets": 3, "reps": 12},
                    {"name": "Lateral Raise", "muscle_group": "Shoulders", "equipment": "Dumbbells", "sets": 3, "reps": 12},
                    {"name": "Bicep Curl", "muscle_group": "Arms", "equipment": "Dumbbells", "sets": 3, "reps": 12},
                    {"name": "Overhead Tricep Extension", "muscle_group": "Arms", "equipment": "Dumbbells", "sets": 3, "reps": 12},
                ],
                sets_and_reps=["3 sets of 12 reps for each exercise"],
                rest_times=["60 seconds between sets", "90 seconds between exercises"],
                notes="Start with a 5-minute warm-up. Use a weight that challenges you by the last rep. Focus on proper form rather than heavy weight."
            )
        )
    ]
    return examples

# Workout tracking and history
class WorkoutTracker:
    def __init__(self, db_path='data/workouts.db'):
        """Initialize the workout tracker database."""
        os.makedirs(os.path.dirname(db_path), exist_ok=True)
        
        self.conn = sqlite3.connect(db_path)
        self.conn.row_factory = sqlite3.Row
        self.cursor = self.conn.cursor()
        self._create_tables()
    
    def _create_tables(self):
        """Create the necessary tables if they don't exist."""
        # Create workouts table
        self.cursor.execute('''
        CREATE TABLE IF NOT EXISTS workouts (
            id INTEGER PRIMARY KEY,
            title TEXT NOT NULL,
            description TEXT,
            date TEXT NOT NULL,
            gym_id INTEGER,
            workout_data TEXT NOT NULL
        )
        ''')
        
        # Create workout_logs table for tracking sets, reps, weights
        self.cursor.execute('''
        CREATE TABLE IF NOT EXISTS workout_logs (
            id INTEGER PRIMARY KEY,
            workout_id INTEGER NOT NULL,
            exercise_name TEXT NOT NULL,
            set_number INTEGER NOT NULL,
            reps INTEGER,
            weight REAL,
            rest_time INTEGER,
            notes TEXT,
            timestamp TEXT NOT NULL,
            FOREIGN KEY (workout_id) REFERENCES workouts (id)
        )
        ''')
        
        self.conn.commit()
    
    def save_workout(self, title, description, gym_id, workout_data):
        """Save a workout plan to the database."""
        date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        
        # Convert workout data to JSON string
        if isinstance(workout_data, dict):
            workout_data = json.dumps(workout_data)
        
        self.cursor.execute(
            'INSERT INTO workouts (title, description, date, gym_id, workout_data) VALUES (?, ?, ?, ?, ?)',
            (title, description, date, gym_id, workout_data)
        )
        self.conn.commit()
        return self.cursor.lastrowid
    
    def log_exercise_set(self, workout_id, exercise_name, set_number, reps=None, weight=None, rest_time=None, notes=None):
        """Log a completed exercise set."""
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        
        self.cursor.execute(
            'INSERT INTO workout_logs (workout_id, exercise_name, set_number, reps, weight, rest_time, notes, timestamp) VALUES (?, ?, ?, ?, ?, ?, ?, ?)',
            (workout_id, exercise_name, set_number, reps, weight, rest_time, notes, timestamp)
        )
        self.conn.commit()
        return self.cursor.lastrowid
    
    def get_workout(self, workout_id):
        """Get a workout by ID."""
        self.cursor.execute('SELECT * FROM workouts WHERE id = ?', (workout_id,))
        workout = self.cursor.fetchone()
        if workout:
            result = dict(workout)
            result['workout_data'] = json.loads(result['workout_data'])
            return result
        return None
    
    def get_recent_workouts(self, limit=10):
        """Get recent workouts."""
        self.cursor.execute('SELECT * FROM workouts ORDER BY date DESC LIMIT ?', (limit,))
        workouts = self.cursor.fetchall()
        return [dict(w) for w in workouts]
    
    def get_workout_logs(self, workout_id):
        """Get all logs for a specific workout."""
        self.cursor.execute('SELECT * FROM workout_logs WHERE workout_id = ? ORDER BY exercise_name, set_number', (workout_id,))
        logs = self.cursor.fetchall()
        return [dict(log) for log in logs]
    
    def close(self):
        """Close the database connection."""
        self.conn.close()

In [3]:
def new_workout():
    """Create a new workout."""
    if request.method == 'POST':
        # Save selected preferences to session
        session['model_provider'] = request.form.get('model_provider', 'openai')
        session['gym_id'] = request.form.get('gym_id')
        workout_description = request.form.get('workout_description', '')
        
        # Get gym equipment if a gym was selected
        gym_equipment = []
        if session['gym_id'] and session['gym_id'] != 'none':
            gym_db = GymDB()
            equipment = gym_db.get_gym_equipment(int(session['gym_id']))
            gym_db.close()
            
            # Format equipment for the model
            gym_equipment = [{
                'name': item['name'],
                'category': item['category'],
                'quantity': item['quantity']
            } for item in equipment]
        
        try:
            # Configure LLM
            dspy.settings.configure(lm=configure_lm(session['model_provider']))
            
            # Create and optimize the workout generator
            workout_generator = WorkoutGenerator()
            
            # Bootstrap with examples for better performance
            examples = bootstrap_examples()
            teleprompter = dspy.teleprompt.BootstrapFewShot(metric=dspy.evaluate.answer_exact_match)
            optimized_generator = teleprompter.compile(
                workout_generator,
                trainset=examples,
                num_bootstrapped_examples=2
            )
            
            # Generate the workout plan
            workout_plan = optimized_generator(workout_description, gym_equipment)
            
            # Save to session for the confirm step
            session['workout_plan'] = {
                'title': workout_plan.title,
                'description': workout_plan.description,
                'exercises': workout_plan.exercises,
                'sets_and_reps': workout_plan.sets_and_reps,
                'rest_times': workout_plan.rest_times,
                'notes': workout_plan.notes
            }
            
            return redirect(url_for('confirm_workout'))
        
        except Exception as e:
            return render_template('error.html', error=str(e))
    
    # Get available gyms for the form
    gym_db = GymDB()
    gyms = gym_db.get_all_gyms()
    gym_db.close()
    
    return render_template('new_workout.html', 
                          gyms=gyms,
                          api_key_status={
                              'openai': bool(os.environ.get('OPENAI_API_KEY')),
                              'claude': bool(os.environ.get('ANTHROPIC_API_KEY'))
                          })

In [4]:
model_provider = "openai"

In [5]:
gym_db = GymDB()

In [6]:
gym_db.get_all_gyms()

[{'id': 1,
  'name': 'Home Gym',
  'location': 'Home',
  'description': 'Basic garage gym with only free weights, squat rack, and cables'}]

In [7]:
gym_id = 1

In [8]:
workout_description = "1 hour long hypertrophy based leg workout"

In [9]:
gym_equipment = []
equipment = gym_db.get_gym_equipment(int(gym_id))
gym_db.close()

# Format equipment for the model
gym_equipment = [{
    'name': item['name'],
    'category': item['category'],
    'quantity': item['quantity']
} for item in equipment]

In [10]:
class WorkoutRequest(dspy.Signature):
    """A request for a workout."""
    workout_request: str = dspy.InputField()
    gym_equipment: List[Dict] = dspy.InputField()
    available_exercises: List[Dict] = dspy.InputField()
    title: str = dspy.OutputField()
    description: str = dspy.OutputField()
    exercises: List[Dict[str, Any]] = dspy.OutputField()
    sets_and_reps: List[str] = dspy.OutputField()
    rest_times: List[str] = dspy.OutputField()
    notes: str = dspy.OutputField()

In [14]:
# Configure LLM
dspy.settings.configure(lm=configure_lm(model_provider))

# Create and optimize the workout generator
workout_generator = WorkoutGenerator()

# Bootstrap with examples for better performance
examples = bootstrap_examples()
teleprompter = dspy.teleprompt.BootstrapFewShot(metric=dspy.evaluate.answer_exact_match)
optimized_generator = teleprompter.compile(
    workout_generator,
    trainset=examples,
)

# Generate the workout plan
workout_plan = optimized_generator(workout_description, gym_equipment)

2025/03/02 12:05:37 ERROR dspy.teleprompt.bootstrap: Failed to run or to evaluate example Example({}) (input_keys=None) with <function answer_exact_match at 0x12fdf0ae0> due to Inputs have not been set for this example. Use `example.with_inputs()` to set them..
2025/03/02 12:05:37 ERROR dspy.teleprompt.bootstrap: Failed to run or to evaluate example Example({}) (input_keys=None) with <function answer_exact_match at 0x12fdf0ae0> due to Inputs have not been set for this example. Use `example.with_inputs()` to set them..
100%|█████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 567.68it/s]

Bootstrapped 0 full traces after 1 examples for up to 1 rounds, amounting to 2 attempts.
1 hour long hypertrophy based leg workout


ValidationError: 6 validation errors for WorkoutRequest
title
  Field required [type=missing, input_value={'workout_request': '1 ho...ment': 'Weight Plate'}]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
description
  Field required [type=missing, input_value={'workout_request': '1 ho...ment': 'Weight Plate'}]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
exercises
  Field required [type=missing, input_value={'workout_request': '1 ho...ment': 'Weight Plate'}]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
sets_and_reps
  Field required [type=missing, input_value={'workout_request': '1 ho...ment': 'Weight Plate'}]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
rest_times
  Field required [type=missing, input_value={'workout_request': '1 ho...ment': 'Weight Plate'}]}, input_type=dict]
    For 